# ⚒️ Tool calling via chat model (OpenAI)

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
from pydantic import BaseModel, Field
from langchain_core.tools import tool


class InputSchema(BaseModel):
    a: int = Field(description="first number")
    b: int = Field(description="second number")


@tool(args_schema=InputSchema)
def multiply(a: int, b: int):
    """Multiply two numbers"""

    return a * b


@tool(args_schema=InputSchema)
def add(a: int, b: int):
    """Add two numbers"""

    return a + b

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

tools = [multiply, add]
tool_by_name = {tool.name: tool for tool in tools}
llm_with_tools = llm.bind_tools(tools)

{'multiply': StructuredTool(name='multiply', description='Multiply two numbers', args_schema=<class '__main__.InputSchema'>, func=<function multiply at 0x00000200C75E99E0>), 'add': StructuredTool(name='add', description='Add two numbers', args_schema=<class '__main__.InputSchema'>, func=<function add at 0x00000200C75E9F80>)}


In [7]:
query = "what is 2 multiply by 5. then add by 5"

print(llm_with_tools.invoke(query).tool_calls)

[{'name': 'multiply', 'args': {'a': 2, 'b': 5}, 'id': 'call_2FRwjIPOQSZbB5y9JIWsERWp', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 5, 'b': 5}, 'id': 'call_UTla1eAWzA383cldj47biA4z', 'type': 'tool_call'}]


## ⚒️ Pass tool output to chat models

![pass-tool-output-to-chat-models](https://python.langchain.com/assets/images/tool_invocation-7f277888701ee431a17607f1a035c080.png)

![pass-tool-output-to-chat-models](https://python.langchain.com/assets/images/tool_results-71b4b90f33a56563c102d91e7821a993.png)

In [10]:
from langchain_core.messages import HumanMessage

query = "what is 3 * 2? Also, what is 11 + 49"

messages = [HumanMessage(content=query)]

ai_message = llm_with_tools.invoke(messages)

print(ai_message)

messages.append(ai_message)

content='' additional_kwargs={'tool_calls': [{'id': 'call_n7dPtZmrw7IsD0aShBwKRhRH', 'function': {'arguments': '{"a": 3, "b": 2}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_WtoOMhOAwKdvfga0jMFeyncd', 'function': {'arguments': '{"a": 11, "b": 49}', 'name': 'add'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 96, 'total_tokens': 147, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_129a36352a', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-37d38044-dda7-44af-8362-1da3fabf2fd7-0' tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 2}, 'id': 'call_n7dPtZmrw7IsD0aShBwKRhRH', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': 'call_WtoOMhOAwKdvfga0jMFeyncd

In [18]:
for tool_call in ai_message.tool_calls:
    select_tool = tool_by_name[tool_call["name"]]
    tool_msg = select_tool.invoke(tool_call)

    messages.append(tool_msg)

print(messages)

[HumanMessage(content='what is 3 * 2? Also, what is 11 + 49', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_n7dPtZmrw7IsD0aShBwKRhRH', 'function': {'arguments': '{"a": 3, "b": 2}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_WtoOMhOAwKdvfga0jMFeyncd', 'function': {'arguments': '{"a": 11, "b": 49}', 'name': 'add'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 96, 'total_tokens': 147, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_129a36352a', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-37d38044-dda7-44af-8362-1da3fabf2fd7-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 2}, 'id': 'call_n7dPtZmr

In [19]:
llm_with_tools.invoke(messages)

AIMessage(content='The result of \\(3 \\times 2\\) is \\(6\\), and \\(11 + 49\\) equals \\(60\\).', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 162, 'total_tokens': 193, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_129a36352a', 'finish_reason': 'stop', 'logprobs': None}, id='run-831e1509-dc9b-4ebf-bfaf-f9d825281c2c-0', usage_metadata={'input_tokens': 162, 'output_tokens': 31, 'total_tokens': 193, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})